In [4]:
pip install scikit-learn


     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     ---------------------------------------- 0.1/11.0 MB 1.3 MB/s eta 0:00:09
      --------------------------------------- 0.2/11.0 MB 1.7 MB/s eta 0:00:07
     - -------------------------------------- 0.5/11.0 MB 2.5 MB/s eta 0:00:05
     --- ------------------------------------ 0.9/11.0 MB 3.8 MB/s eta 0:00:03
     ----- ---------------------------------- 1.5/11.0 MB 5.4 MB/s eta 0:00:02
     --------- ------------------------------ 2.6/11.0 MB 7.8 MB/s eta 0:00:02
     ------------------ --------------------- 5.1/11.0 MB 13.6 MB/s eta 0:00:01
     ------------------------ --------------- 6.8/11.0 MB 16.1 MB/s eta 0:00:01
     ------------------------ --------------- 6.8/11.0 MB 16.1 MB/s eta 0:00:01
     ---------------------------- ----------- 7.8/11.0 MB 14.7 MB/s eta 0:00:01
     ------------------------------------ --- 9.9/11.0 MB 17.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold, RFE, SelectFromModel
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier

In [7]:
df = pd.read_csv('D:/features_selection\data_sample\LoanDataset - LoansDatasest.csv')
df.info()
# Tìm dữ liệu lỗi
df.isna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32586 entries, 0 to 32585
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          32583 non-null  float64
 1   customer_age         32586 non-null  int64  
 2   customer_income      32586 non-null  object 
 3   home_ownership       32586 non-null  object 
 4   employment_duration  31691 non-null  float64
 5   loan_intent          32586 non-null  object 
 6   loan_grade           32586 non-null  object 
 7   loan_amnt            32585 non-null  object 
 8   loan_int_rate        29470 non-null  float64
 9   term_years           32586 non-null  int64  
 10  historical_default   11849 non-null  object 
 11  cred_hist_length     32586 non-null  int64  
 12  Current_loan_status  32582 non-null  object 
dtypes: float64(3), int64(3), object(7)
memory usage: 3.2+ MB


,customer_id,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32581,False,False,False,False,False,False,False,False,False,False,True,False,False
32582,False,False,False,False,False,False,False,False,False,False,True,False,False
32583,False,False,False,False,False,False,False,False,False,False,False,False,False
32584,False,False,False,False,False,False,False,False,False,False,True,False,False


In [ ]:
df.drop['employment_duration','']